In [ ]:
#Constraints For The Given Problem

"""
Hard Constraints (10 Points Each)
• An exam will be scheduled for each course.
• A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a time.
• Exam will not be held on weekends.
• Each exam must be held between 9 am and 5 pm
• Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.
• A teacher cannot invigilate two exams in a row.

Soft Constraints (5 Points Each)
• All students and teachers shall be given a break on Friday from 1-2.
• A student shall not give more than 1 exam consecutively.
• Two hours of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot so the faculty meetings shall be held in parts as they are now.
• If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam.

Additional Constraints (Mentioned Later on, Points not known)
• 28 Students per Class
• Use binary encoding for chromosome.

"""

#Deliverables

"""
.pynb File
One Page PDF Report
"""


In [89]:
# %%
# All Imports

import csv
import math
import numpy as np

In [118]:
#Hyper Parameters

population_size = 100
crossover_rate = 0.5

#Generic Variables
max_students_per_class = 28

days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
total_timeslots = len(days)*2
morning_starting = '9AM'
morning_ending = '12PM'
evening_starting = '2PM'
evening_ending = '5PM'

student_data = None
teacher_data = None
course_data = None
student_in_course = None
courses_to_student = None
timeslots = None

In [119]:
class chromosome:
    courses = {}
    #Intialize courses dictionary, course is a key, and value is 2d list, 
    #first row of list contains timeslot, second contains teachers
    def __init__(self):
        for key in course_data.keys():
            array_timeslots = np.zeros(total_timeslots, dtype = int)
            array_teachers = np.zeros(len(teacher_data), dtype = int)
            self.courses[key] = [array_timeslots, array_teachers]#np.vstack((array_timeslots,array_teachers)).T
    #Get element from timeslot of teacher        
    def getValue(self,course_code, array, element):
        return self.courses[course_code][array][element]
    #Set element of timeslot or teacher
    def setValue(self,course_code, array, element, value):
        self.courses[course_code][array][element] = value

In [120]:
#Data Extraction from Files
def read_student_data(filename):
    #Storing student data in dictionary
    #Key is 0,1,2,3 ; value is student name
    student_data = dict()
    count = 0
    with open(filename,mode = 'r') as student_datafile:
        read = csv.reader(student_datafile,delimiter = ',')
        for name in read:
            #studentData.add(name[0])
            student_data[count] = name[0]
            count += 1
    #print(studentData)
    return student_data
def read_teacher_data(filename):
    #Storing teacher data in dictionary
    #key is 0,1,2,3 ; value is teacher names
    teacher_data = dict()
    count = 0
    with open(filename,mode = 'r') as teacher_datafile:
        read = csv.reader(teacher_datafile,delimiter = ',')
        for name in read:
            if len(name) == 0:
                continue
            teacher_data[count] = name[0]
            count += 1
    #print(teacherData)
    return teacher_data
def read_course_data(filename):
    #Storing Course data
    #Key is course code, value is course name
    courses_data = dict()
    count = 0
    with open(filename,mode = 'r') as courses_datafile:
        read = csv.reader(courses_datafile,delimiter = ',')
        for name in read:
            courses_data[name[0]] = name[1]
    #print(coursesData)
    return courses_data
def read_student_in_course_data(filename, courses_data, student_data):
    #Storing student in each course
    #Key is course; value is list of studen taking course
    student_in_course_data = dict()
    for i in courses_data:
        student_in_course_data[i] = []
    count = 0
    student_value_list = list(student_data.values())
    with open(filename, mode = 'r') as student_in_course_datafile:
        read = csv.reader(student_in_course_datafile, delimiter = ',')
        for name in read:
            #Headers in studentCourse.csv
            if ( count == 0 ):
                count += 1
                continue
            student_in_course_data[name[2]].append(student_value_list.index(name[1]))
            count += 1
    return student_in_course_data
def courses_to_student_data(filename, student_data):
    #Storing courses taken by each student
    #Key is student; value is list of courses taken by student
    courses_to_student_data = dict()
    for i in student_data:
        courses_to_student_data[i] = []
    count = 0
    student_value_list = list(student_data.values())
    with open(filename, mode = 'r') as courses_to_student_datafile:
        read = csv.reader(courses_to_student_datafile, delimiter = ',')
        for name in read:
            #Headers in studentCourse.csv
            if ( count == 0 ):
                count += 1
                continue
            courses_to_student_data[student_value_list.index(name[1])].append(name[2])
            count += 1
    #print(courses_to_student_data)
    return courses_to_student_data

def create_timeslots():
    global timeslots
    global days
    global total_timeslots
    global morning_starting
    global morning_ending
    global evening_starting
    global evening_ending
    
    timeslots = dict()
    for i in range(total_timeslots):
        start_time = None
        end_time = None
        if i%2 == 0:
            start_time = morning_starting
            end_time = morning_ending
        else:
            start_time = evening_starting
            end_time = evening_ending
        timeslots[i] = [start_time, end_time, days[int(i/2)]]

In [121]:
#Miscellaneous Functions

#Calculates the Average of Students in Each Course
def avg_students_per_course():
    global student_in_course
    sum_students = 0
    leng = 0
    for i in student_in_course.keys():
        sum_students += len(student_in_course[i])
        leng += 1
    return (sum_students / leng)

#Calculates the number of teachers required for Invigilating the given Course Code (28 Students Max Per Teacher)
def teachers_needed(course_code):
    global student_in_course
    students = student_in_course[course_code]
    num_students = len(students)
    return math.ceil(num_students/max_students_per_class)

#Calculates the Number of Students Registered in the given Course Code
def students_count(course_code):
    global student_in_course
    return len(student_in_course[course_code])
    

In [122]:
#Implementation of Chromosomes here

In [123]:
if __name__ == "__main__":
    global student_data
    global teacher_data
    global coruse_data
    global student_in_course
    global courses_to_student
    global timeslots
    
    #Initialization
    student_data = read_student_data('studentNames.csv')
    teacher_data = read_teacher_data('teachers.csv')
    course_data = read_course_data('courses.csv')
    student_in_course = read_student_in_course_data('studentCourse.csv', course_data, student_data)
    courses_to_student = courses_to_student_data('studentCourse.csv', student_data)
    
    avg_students = avg_students_per_course()
    classrooms_per_course = math.ceil(len(teacher_data.keys())/len(course_data.keys()))
    avg_students_per_classroom = math.ceil(avg_students/classrooms_per_course)
    
    
    create_timeslots()    
    for i in range(total_timeslots):
        print(timeslots[i])
    
    
    #Verification
    print(f"Number of Students: {len(student_data.keys())}")
    print(f"Number of Teachers: {len(teacher_data.keys())}")
    print(f"Number of Courses: {len(course_data.keys())}")
    
  
    print(f"Average Students Per Course: {avg_students}")
    print(f"Classrooms Per Course: {classrooms_per_course}")
    print(f"Average Students Per Classroom: {avg_students_per_classroom}")
    
    
    print(f"Teachers required for EE229: {teachers_needed('EE229')}")
    print(f"Students in EE229: {students_count('EE229')}")
    print(f"Teachers required for CS218: {teachers_needed('CS218')}")
    print(f"Students in CS218: {students_count('CS218')}")

    #p = chromosome()
    #print(p.courses)
    #print(p.courses['CS217'][1][0])

['9AM', '12PM', 'Monday']
['2PM', '5PM', 'Monday']
['9AM', '12PM', 'Tuesday']
['2PM', '5PM', 'Tuesday']
['9AM', '12PM', 'Wednesday']
['2PM', '5PM', 'Wednesday']
['9AM', '12PM', 'Thursday']
['2PM', '5PM', 'Thursday']
['9AM', '12PM', 'Friday']
['2PM', '5PM', 'Friday']
Number of Students: 200
Number of Teachers: 63
Number of Courses: 23
Average Students Per Course: 35.65217391304348
Classrooms Per Course: 3
Average Students Per Classroom: 12
Teachers required for EE229: 3
Students in EE229: 64
Teachers required for CS218: 2
Students in CS218: 36
{'CS217': [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])], 'EE227': [array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,